# Textual Inversion with Determined

This notebook generates images from the trained textual inversion models generated with the `detsd.DetStableDiffusionTITrainer` class and saved as Determined checkpoints.  This notebook should be connected to a GPU.

### Pre-Launch Setup

A [Huggingface User Access Token](https://huggingface.co/docs/hub/security-tokens) is required to download the [Stable Diffusion weights](https://huggingface.co/CompVis/stable-diffusion-v1-4). To use this notebook, please modify the following lines in the `detsd-notebook.yaml` file:
```yaml
environment:
    environment_variables:
        - HF_AUTH_TOKEN=YOUR_HF_AUTH_TOKEN_HERE
```
after which this notebook can be launched by calling the below from the root of the repo directory
```bash
det -m MASTER_URL_WITH_PORT notebook start --config detsd-notebook.yaml --context .
```
and then opening up the copy of `textual_inversion.ipynb` on the master.
**TODO: Update `--context` arg, don't pull in whole repo unnecessarily**

Update the jupyter notebook for better progress-bar rendering. Assuming the notebook has dependencies have already been installed via `startup-hook.sh`, which is run upon agent start-up.

In [1]:
! pip install -qq jupyterlab-widgets==1.1.1 ipywidgets==7.7.2

Import the `DetSDTextualInversionPipeline` class from `detsd.py` (loaded via the `--context flag above), which will be used to generate Stable Diffusion images.

In [ ]:
from detsd import DetSDTextualInversionPipeline

In [ ]:
# Code for logging into the master, if not already logged in.
# Not required if notebook was launched as described above.
# client.login(master: MASTER_URL, user: USER, password: PASS)

## Load Determined Checkpoints

We will now construct the `DetSDTextualInversionPipeline`, incorporating into the model any textual-inversion checkpoints we have trained with `DetStableDiffusionTITrainer`, also contained in `detsd.py`.  These Determined checkpoints can be specified by their uuid, assuming all such checkpoints exist on the master we are currently logged into.

In [11]:
uuids = ['b8e4e741-a770-4359-a301-679b86e5f1f2']
detsd_pipeline = DetSDTextualInversionPipeline.from_uuids(uuids,)

['learned_embeds.pt', 'metadata.json', 'optimizer_state_dict.pt']

The above pipeline was constructed using the default settings. In particular, it uses the default `fp16=True` and `autocast=True` options which are used to increase inference speed and reduce memory usage, at the cost of somewhat reduced-quality images.  Other available args can be viewed by uncommenting and running the cell below

In [ ]:
# ? DetSDTextualInversionPipeline.from_uuids

If you have not trained any textual inversion checkpoints and simply want to generate images using the original Stable Diffusion model without custom concepts added, you can leave the `uuids` list below empty or alternatively instantiate the pipeline via
```python
detsd_pipeline = DetSDTextualInversionPipeline()
```
which uses the same default arguments as above.

## Generate Images

Images can be generated by calling our `detsd_pipeline` instance as in the below:

In [ ]:
# Create a directory in which to save the generated images:
! mkdir generated_images

In [ ]:
# Print out the concepts we have loaded into the pipeline:
detsd_pipeline.concepts
first_concept = detsd_pipeline.concepts[0]

In [ ]:
imgs = detsd_pipeline(prompt=f'a watercolor painting of a {first_concept}, soft strokes, pastel colors',
                      parallelize_factor=2,
                      rows=2,
                      cols=2,
                      num_inference_steps=50,
                      seed=2147483647,
                      guidance_scale=7.5,
                      saved_img_dir='generated_images')
imgs

The above generates `num_samples = rows * cols` total samples, tiling them together into a single image.  We generate `parallelize_factor` such samples at a time; reduce `parallelize_factor` to `1` if you are running out of memory.

The three args below represent your primary knobs for altering the generated output:
* `num_inference_steps`: how many steps to run the generation process for. ~50 is typical
* `guidance_scale`: tunes how much weight is given to the prompt during generation. 7.5 is the default, with larger numbers leading to stronger adherence to the prompt.
* `generator_seed`: fixed RNG seed for reproducibility

Additional arguments can be passed to the underlying Huggingface `StableDiffusionPipeline` instance through the `other_pipeline_call_kwargs` arg. See the [Hugging Face documentation](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion#diffusers.StableDiffusionPipeline.__call__) for information on all available arguments.